In [3]:
#Importando libs
from art.attacks.extraction import CopycatCNN, KnockoffNets
from art.estimators.classification import KerasClassifier
import numpy as np
from glob import glob
import scipy
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import matplotlib.pyplot as plt
from random import randint
from tqdm import tqdm
import pandas as pd
import cv2
from PIL import Image
from statistics import mode
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import metrics, model_selection
from keras.models import Sequential, load_model
import seaborn as sns
import pickle
from sklearn.metrics import roc_auc_score

In [4]:
googl = False

try:
    from google.colab import drive
    drive.mount('/content/drive')
    googl = True
except:
    googl = False

Mounted at /content/drive


In [5]:
if googl:
    infile = open('/content/drive/MyDrive/Lab7sds/dataframe_p3.pkl','rb')
    df = pickle.load(infile)
    infile.close()
else:
    infile = open('dataframe_p3.pkl','rb')
    df = pickle.load(infile)
    infile.close()


Separando Data

In [6]:
target = df['Label_arrays'].to_numpy()
feature_matrix = df['Image'].to_numpy()

print(type(target))
print(type(feature_matrix))

#print('Final features:', feature_matrix.columns)

#División de datos de entrenamientos y prueba
X_train, X_score, y_train, y_score = model_selection.train_test_split(feature_matrix/255, 
                                                                      target, 
                                                                      test_size=0.30, 
                                                                      random_state=31)

X_train.shape

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


(2247,)

In [7]:
#Recasting

X_train = np.array(X_train.tolist(), dtype=float)
X_score = np.array(X_score.tolist(), dtype=float)
y_train = np.array(y_train.tolist(), dtype=float)
y_score = np.array(y_score.tolist(), dtype=float)

'''#Expandiendo dims
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_score, axis=2)'''

print(X_train.shape, type(X_train))

(2247, 128, 128, 3) <class 'numpy.ndarray'>


In [8]:
#Corrigiendo dims
print(y_train.shape)
print(X_train.shape)
print(X_score.shape)
print(y_score.shape)
num_classes=y_train.shape
number_pix=X_train.shape[1]*X_train.shape[2]
print(num_classes)
print(number_pix)

(2247, 23)
(2247, 128, 128, 3)
(964, 128, 128, 3)
(964, 23)
(2247, 23)
16384


Cargando Modelo

In [9]:
if tf.executing_eagerly():
    tf.compat.v1.disable_eager_execution()

if googl:
    malign_model = load_model('/content/drive/MyDrive/Lab7sds/benign_model.h5')
else:
    malign_model = load_model('benign_model.h5')

Ataque de evasión

In [10]:
from art.attacks.evasion import FastGradientMethod

In [11]:
classifier = KerasClassifier(malign_model, clip_values=(0, 1), use_logits=False)

In [13]:
# Craft adversarial samples with FGSM
epsilon = 0.1  # Maximum perturbation
adv_crafter = FastGradientMethod(classifier, eps=epsilon)
x_test_adv = adv_crafter.generate(x=X_score)

In [14]:
# Evaluate the classifier on the adversarial examples
preds = np.argmax(classifier.predict(x_test_adv), axis=1)
acc = np.sum(preds == np.argmax(y_score, axis=1)) / y_score.shape[0]
print("\nTest accuracy on adversarial sample: %.2f%%" % (acc * 100))


Test accuracy on adversarial sample: 10.79%
